# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9204ca15b0>
├── 'a' --> tensor([[-2.2184,  0.9926, -0.1057],
│                   [-0.4250, -2.2181, -0.5031]])
└── 'x' --> <FastTreeValue 0x7f9204ca1d00>
    └── 'c' --> tensor([[-0.1600,  1.9821,  0.6223, -0.8881],
                        [ 0.0814,  2.0722, -0.9121,  0.5785],
                        [ 0.0909,  1.6903,  0.3983,  0.4189]])

In [4]:
t.a

tensor([[-2.2184,  0.9926, -0.1057],
        [-0.4250, -2.2181, -0.5031]])

In [5]:
%timeit t.a

67.2 ns ± 0.0146 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9204ca15b0>
├── 'a' --> tensor([[-0.4733,  0.4861, -1.0426],
│                   [-0.3384, -0.3451,  0.4283]])
└── 'x' --> <FastTreeValue 0x7f9204ca1d00>
    └── 'c' --> tensor([[-0.1600,  1.9821,  0.6223, -0.8881],
                        [ 0.0814,  2.0722, -0.9121,  0.5785],
                        [ 0.0909,  1.6903,  0.3983,  0.4189]])

In [7]:
%timeit t.a = new_value

60 ns ± 0.034 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-2.2184,  0.9926, -0.1057],
               [-0.4250, -2.2181, -0.5031]]),
    x: Batch(
           c: tensor([[-0.1600,  1.9821,  0.6223, -0.8881],
                      [ 0.0814,  2.0722, -0.9121,  0.5785],
                      [ 0.0909,  1.6903,  0.3983,  0.4189]]),
       ),
)

In [10]:
b.a

tensor([[-2.2184,  0.9926, -0.1057],
        [-0.4250, -2.2181, -0.5031]])

In [11]:
%timeit b.a

53 ns ± 0.055 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3452, -0.5744,  0.3755],
               [-0.8024, -0.6832, -0.6237]]),
    x: Batch(
           c: tensor([[-0.1600,  1.9821,  0.6223, -0.8881],
                      [ 0.0814,  2.0722, -0.9121,  0.5785],
                      [ 0.0909,  1.6903,  0.3983,  0.4189]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.108 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

832 ns ± 0.326 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 63.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 417 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 321 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9158992eb0>
├── 'a' --> tensor([[[-2.2184,  0.9926, -0.1057],
│                    [-0.4250, -2.2181, -0.5031]],
│           
│                   [[-2.2184,  0.9926, -0.1057],
│                    [-0.4250, -2.2181, -0.5031]],
│           
│                   [[-2.2184,  0.9926, -0.1057],
│                    [-0.4250, -2.2181, -0.5031]],
│           
│                   [[-2.2184,  0.9926, -0.1057],
│                    [-0.4250, -2.2181, -0.5031]],
│           
│                   [[-2.2184,  0.9926, -0.1057],
│                    [-0.4250, -2.2181, -0.5031]],
│           
│                   [[-2.2184,  0.9926, -0.1057],
│                    [-0.4250, -2.2181, -0.5031]],
│           
│                   [[-2.2184,  0.9926, -0.1057],
│                    [-0.4250, -2.2181, -0.5031]],
│           
│                   [[-2.2184,  0.9926, -0.1057],
│                    [-0.4250, -2.2181, -0.5031]]])
└── 'x' --> <FastTreeValue 0x7f9158992df0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.5 µs ± 86.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f91589750a0>
├── 'a' --> tensor([[-2.2184,  0.9926, -0.1057],
│                   [-0.4250, -2.2181, -0.5031],
│                   [-2.2184,  0.9926, -0.1057],
│                   [-0.4250, -2.2181, -0.5031],
│                   [-2.2184,  0.9926, -0.1057],
│                   [-0.4250, -2.2181, -0.5031],
│                   [-2.2184,  0.9926, -0.1057],
│                   [-0.4250, -2.2181, -0.5031],
│                   [-2.2184,  0.9926, -0.1057],
│                   [-0.4250, -2.2181, -0.5031],
│                   [-2.2184,  0.9926, -0.1057],
│                   [-0.4250, -2.2181, -0.5031],
│                   [-2.2184,  0.9926, -0.1057],
│                   [-0.4250, -2.2181, -0.5031],
│                   [-2.2184,  0.9926, -0.1057],
│                   [-0.4250, -2.2181, -0.5031]])
└── 'x' --> <FastTreeValue 0x7f91dc44fb50>
    └── 'c' --> tensor([[-0.1600,  1.9821,  0.6223, -0.8881],
                        [ 0.0814,  2.0722, -0.9121,  0.5785],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 18 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.6 µs ± 45.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-2.2184,  0.9926, -0.1057],
                [-0.4250, -2.2181, -0.5031]],
       
               [[-2.2184,  0.9926, -0.1057],
                [-0.4250, -2.2181, -0.5031]],
       
               [[-2.2184,  0.9926, -0.1057],
                [-0.4250, -2.2181, -0.5031]],
       
               [[-2.2184,  0.9926, -0.1057],
                [-0.4250, -2.2181, -0.5031]],
       
               [[-2.2184,  0.9926, -0.1057],
                [-0.4250, -2.2181, -0.5031]],
       
               [[-2.2184,  0.9926, -0.1057],
                [-0.4250, -2.2181, -0.5031]],
       
               [[-2.2184,  0.9926, -0.1057],
                [-0.4250, -2.2181, -0.5031]],
       
               [[-2.2184,  0.9926, -0.1057],
                [-0.4250, -2.2181, -0.5031]]]),
    x: Batch(
           c: tensor([[[-0.1600,  1.9821,  0.6223, -0.8881],
                       [ 0.0814,  2.0722, -0.9121,  0.5785],
                       [ 0.0909,  1.6903,  0.3983,  0.4189]],
         

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 296 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-2.2184,  0.9926, -0.1057],
               [-0.4250, -2.2181, -0.5031],
               [-2.2184,  0.9926, -0.1057],
               [-0.4250, -2.2181, -0.5031],
               [-2.2184,  0.9926, -0.1057],
               [-0.4250, -2.2181, -0.5031],
               [-2.2184,  0.9926, -0.1057],
               [-0.4250, -2.2181, -0.5031],
               [-2.2184,  0.9926, -0.1057],
               [-0.4250, -2.2181, -0.5031],
               [-2.2184,  0.9926, -0.1057],
               [-0.4250, -2.2181, -0.5031],
               [-2.2184,  0.9926, -0.1057],
               [-0.4250, -2.2181, -0.5031],
               [-2.2184,  0.9926, -0.1057],
               [-0.4250, -2.2181, -0.5031]]),
    x: Batch(
           c: tensor([[-0.1600,  1.9821,  0.6223, -0.8881],
                      [ 0.0814,  2.0722, -0.9121,  0.5785],
                      [ 0.0909,  1.6903,  0.3983,  0.4189],
                      [-0.1600,  1.9821,  0.6223, -0.8881],
                      [ 0.0814,  

In [28]:
%timeit Batch.cat(batches)

143 µs ± 311 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
